<a href="https://colab.research.google.com/github/thmswhelan/GAA_Rankings/blob/main/Gaa_Football_Rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import csv

# URL for GAA fixtures or competition page
url = "https://www.gaa.ie/fixtures-results"

# Send HTTP request to fetch the page content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    page_content = response.content
    soup = BeautifulSoup(page_content, "html.parser")

    # Prepare CSV file to save the data
    with open('gaa_matches.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write headers to the CSV
        writer.writerow(["Competition", "Home Team", "Away Team", "Date", "Venue", "Home Score", "Away Score"])

        # Extract match details
        match_details = soup.find_all("div", class_="gar-match-item")

        for match in match_details:
            # Example of extracting team names
            home_team = match.find("div", class_="gar-match-item__team -home")
            away_team = match.find("div", class_="gar-match-item__team -away")

            # Extract the competition name for each match (assuming it may be inside the match item)
            competition = match.find_previous("h3", class_="gar-matches-list__group-name")
            competition_name = competition.get_text(strip=True) if competition else "Unknown Competition"

            # Extract the date of the match
            match_date = match.get("data-match-date", "No date available")

            # Extract the venue for the match
            venue = match.find("div", class_="gar-match-item__venue")
            venue_name = venue.get_text(strip=True) if venue else "No venue available"

            # Extract the home and away scores
            scores = match.find_all("div", class_="gar-match-item__score")

            # If scores are found, we process them correctly
            if len(scores) == 2:
                home_score = scores[0].get_text(strip=True)
                away_score = scores[1].get_text(strip=True)
            else:
                home_score = "N/A"
                away_score = "N/A"

            # If the score contains a date, we make sure to extract only the score
            if '-' not in home_score or '-' not in away_score:
                home_score = "Invalid score"
                away_score = "Invalid score"

            # Get team names
            if home_team and away_team:
                home_team_name = home_team.get_text(strip=True)
                away_team_name = away_team.get_text(strip=True)
            else:
                home_team_name = "Unknown"
                away_team_name = "Unknown"

            # Write each match's data to the CSV file
            writer.writerow([competition_name, home_team_name, away_team_name, match_date, venue_name, home_score, away_score])

    print("Data saved to 'gaa_matches.csv'.")
else:
    print("Failed to retrieve the page. Status Code:", response.status_code)



Data saved to 'gaa_matches.csv'.
